<a href="https://colab.research.google.com/github/mille055/Rec_Project/blob/main/notebooks/Rescrape_using_siddgood_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
!pip install unidecode selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [118]:
from bs4 import BeautifulSoup
import requests
import json

import pandas as pd
import numpy as np

import json
import re
import unidecode
import time
from tqdm import tqdm

from selenium import webdriver



In [119]:
CATEGORIES = {
    'arts': "https://podcasts.apple.com/us/genre/podcasts-arts/id1301",
    'business': "https://podcasts.apple.com/us/genre/podcasts-business/id1321",
    'comedy': "https://podcasts.apple.com/us/genre/podcasts-comedy/id1303",
    'education': "https://podcasts.apple.com/us/genre/podcasts-education/id1304",
    'fiction': "https://podcasts.apple.com/us/genre/podcasts-fiction/id1483",
    'government': "https://podcasts.apple.com/us/genre/podcasts-government/id1511",
    'health': "https://podcasts.apple.com/us/genre/podcasts-health-fitness/id1512",
    'history': "https://podcasts.apple.com/us/genre/podcasts-history/id1487",
    'kids_and_family': "https://podcasts.apple.com/us/genre/podcasts-kids-family/id1305",
    'leisure': "https://podcasts.apple.com/us/genre/podcasts-leisure/id1502",
    'music': "https://podcasts.apple.com/us/genre/podcasts-music/id1310",
    'news': "https://podcasts.apple.com/us/genre/podcasts-news/id1489",
    'religion_and_spirituality': "https://podcasts.apple.com/us/genre/podcasts-religion-spirituality/id1314",
    'science': "https://podcasts.apple.com/us/genre/podcasts-science/id1533",
    'society_and_culture': "https://podcasts.apple.com/us/genre/podcasts-society-culture/id1324",
    'sports': "https://podcasts.apple.com/us/genre/podcasts-sports/id1545",
    'tv_and_film': "https://podcasts.apple.com/us/genre/podcasts-tv-film/id1309",
    'technology': "https://podcasts.apple.com/us/genre/podcasts-technology/id1318",
    'true_crime': "https://podcasts.apple.com/us/genre/podcasts-true-crime/id1488"
}

In [120]:
all_podcast_links = list()

for category_url in CATEGORIES.values():
    response = requests.get(category_url, timeout=5)
    content = BeautifulSoup(response.content, "lxml")
    podcast_links = content.find('div', class_='grid3-column')

    for link in podcast_links.findAll('a'):
        all_podcast_links.append(link.get('href'))

all_podcast_links = list(set(all_podcast_links))

print('Number of podcast links', len(all_podcast_links))

with open('podcast_links.json', 'w') as outfile:
    json.dump(all_podcast_links, outfile)

Number of podcast links 4560


In [121]:
def convert_si_to_number(x):
    total_stars = 0
    if 'K' in x:
        if len(x) > 1:
            total_stars = float(x.replace('K', '')) * 1000  # convert K to a thousand
    elif 'M' in x:
        if len(x) > 1:
            total_stars = float(x.replace('M', '')) * 1000000  # convert M to a million
    elif 'B' in x:
        total_stars = float(x.replace('B', '')) * 1000000000  # convert B to a Billion
    else:
        total_stars = int(x)  # Less than 1000
    return int(total_stars)

In [137]:

def build_podcast_object(link):
    try:
        response = requests.get(link, timeout=10)
        content = BeautifulSoup(response.content, "lxml")
        try:
            title = content.find('h1').find('span').text.strip()
        except AttributeError:
            title = 'NA'
        try:
            producer = content.find('h1').find('a').text.strip()
        except AttributeError:
            try:
                producer = content.find('h1').find('span', class_='product-header__identity podcast-header__identity') \
                    .text.strip()
            except AttributeError:
                producer = 'NA'
        try:
            genre = content.find('li', class_="product-header__list__item").text.strip()
        except AttributeError:
            genre = 'NA'
        try:
            description = content.find('div', class_="product-hero-desc product-hero-desc--side-bar").text.strip()
        except AttributeError:
            description = 'NA'
        try:
            num_episodes = int(content.find('div', class_="product-artwork__caption small-hide medium-show")
                               .text.strip().strip('episodes'))
        except AttributeError:
            num_episodes = 'NA'
        try:
            rating = float(content.find('span', class_="we-customer-ratings__averages__display").text.strip())
        except AttributeError:
            rating = 'NA'
        try:
            num_reviews = convert_si_to_number(content
                                               .find('div', class_="we-customer-ratings__count small-hide medium-show")
                                               .text.strip().strip('Ratings'))
        except AttributeError:
            num_reviews = 'NA'

        try:
            script_tag = content.find('script', {'name': 'schema:podcast-show', 'type': 'application/ld+json'})

            # Extract and load the JSON data
            json_data = json.loads(script_tag.string)

            # Access the features in the JSON data
            script = {'jname': json_data['name'], 
                      'jauthor': json_data['author'],
                      'jdescription': json_data['description'],        
                      'jdate_published': json_data['datePublished'],
                      'joffers': json_data['offers'],
                      'jreviews': json_data['review'],
                      'work_examples' : json_data['workExample']}


        except AttributeError:
            script = 'NA'

    except Exception:
        title = 'NA'
        producer = 'NA'
        genre = 'NA'
        description = 'NA'
        num_episodes = 'NA'
        rating = 'NA'
        num_reviews = 'NA'
        script = 'NA'
    podcast_object = {
        'title': title,
        'producer': producer,
        'genre': genre,
        'description': script['jdescription'],
        'num_episodes': num_episodes,
        'rating': rating,
        'num_reviews': num_reviews,
        'link': link,
        'itunes_id': link.split('/')[-1].strip('id'),
        'epidsode_descriptions': [x['description'] for x in script['work_examples']]
    }
    return podcast_object


In [140]:
counter = 1
podcastObjects = list()
with open('podcast_links.json') as json_file:
    podcasts = json.load(json_file)
    for podcast_link in podcasts:
        try:
            podcast = build_podcast_object(podcast_link)
            podcastObjects.append(podcast)
            print('Completed Podcast: ' + str(counter))
            counter += 1
        except Exception:
            print('Failed on ' + podcast_link)
            counter += 1
            pass

with open('podcasts_info_with_episodedescriptions.json', 'w') as outfile:
    json.dump(podcastObjects, outfile)

Completed Podcast: 1
Completed Podcast: 2
Completed Podcast: 3
Completed Podcast: 4
Completed Podcast: 5
Completed Podcast: 6
Failed on https://podcasts.apple.com/us/podcast/the-sports-junkies/id367825854
Completed Podcast: 8
Completed Podcast: 9
Completed Podcast: 10
Completed Podcast: 11
Completed Podcast: 12
Completed Podcast: 13
Completed Podcast: 14
Completed Podcast: 15
Completed Podcast: 16
Completed Podcast: 17
Completed Podcast: 18
Completed Podcast: 19
Completed Podcast: 20
Completed Podcast: 21
Completed Podcast: 22
Completed Podcast: 23
Completed Podcast: 24
Completed Podcast: 25
Completed Podcast: 26
Completed Podcast: 27
Failed on https://podcasts.apple.com/us/podcast/the-npr-politics-podcast/id1057255460
Completed Podcast: 29
Completed Podcast: 30
Completed Podcast: 31
Completed Podcast: 32
Completed Podcast: 33
Completed Podcast: 34
Completed Podcast: 35
Completed Podcast: 36
Completed Podcast: 37
Completed Podcast: 38
Completed Podcast: 39
Completed Podcast: 40
Complet

In [147]:
sample_podcast = build_podcast_object(podcast_link)

In [148]:
sample_podcast

{'title': 'Loving Well',
 'producer': 'Focus on the Family',
 'genre': 'Kids & Family',
 'description': 'The secret sauce to the success of your marriage! Every season of this bingeable podcast series is chalk full of heartwarming love stories and practical marriage advice you can put into practice every day. The series from Focus on the Family will inspire you and your spouse to put Christ’s love at t…',
 'num_episodes': 32,
 'rating': 4.6,
 'num_reviews': 185,
 'link': 'https://podcasts.apple.com/us/podcast/loving-well/id1342754239',
 'itunes_id': '1342754239',
 'epidsode_description': ['Learning to see your spouse through the lens of God’s love is one of the best ways to keep your marriage strong.\xa0John talks with Erin and Greg about why being humble is good for your relationship, and how you can appreciate the way God made your mate.\n\nReceive the book "Crazy Little Thing Called Marriage: 12 Secrets for a Lifelong Romance" for your donation of any amount: https://store.focusonth

In [141]:
def clean_title(t):
    t = unidecode.unidecode(t)
    t = t.replace('\n', ' ')
    t = re.sub(r'[^\w\s]', '', t)
    t = re.sub(r'\d+', '', t)
    t = t.lower()
    t = t.strip()
    return t


In [166]:
def clean_description(dlist):
    
    new_list = []
    for d in dlist:
      d = unidecode.unidecode(d)
      d = d.replace('\n', ' ')
      d = re.sub(r'[^\w\s]', '', d)
      d = re.sub(r'\d+', '', d)
      if re.findall(r'(.*) brought to you by.*', d):
        d = re.sub(r'brought to you by.*', '', d)
      if re.search(r'(.*) sponsored by.*', d):
        d = re.sub(r'sponsored by.*', '', d)
      d = d.lower()
      d = d.strip()
      new_list.append(d)
    return new_list


In [114]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
!pip install kora -q

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chro

In [115]:
def get_recent_podcast_episodes(link):
    episode_titles = ''
    episode_desc = ''

    from kora.selenium import wd
    wd.get(link)
    
    html = wd.page_source.encode('utf-8')

    soup = BeautifulSoup(html, 'lxml')
    text = str(soup.find('script'))

    try:
        text = text.split('"workExample":')[1].split(',"aggregateRating"')[0]
        episode_data = json.loads(text)

        for episode in episode_data:
            title = episode['name']
            c_title = clean_title(title)

            description = episode['description']
            c_description = clean_description(description)

            episode_titles += (c_title + " ")
            episode_desc += (c_description + " ")

        episode_titles = episode_titles.strip()
        episode_desc = episode_desc.strip()

    except Exception:
        episode_title = np.nan
        episode_desc = np.nan
        print("Failed on: " + str(link))

    return [episode_titles, episode_desc]

In [149]:
with open('podcasts_info_with_episodedescriptions.json') as json_file:
        podcasts = json.load(json_file)

podcasts_df = pd.DataFrame(podcasts)

podcasts_df = podcasts_df[['title', 'producer', 'genre', 'description', 'num_episodes',
                               'rating', 'num_reviews', 'link', 'itunes_id', 'episode_description']]
podcasts_df = podcasts_df.replace('NA', np.nan)
#podcasts_df = podcasts_df.dropna()

print(podcasts_df.head())

podcast_titles = list(podcasts_df['title'])
podcast_titles = [title.replace(" ", "") for title in podcast_titles]
podcast_titles = [re.sub(r'[^\w\s]', '', title) for title in podcast_titles]
is_english = [bool(re.match("^[A-Za-z9]*$", title)) for title in podcast_titles]
podcasts_df['is_english'] = is_english
podcasts_df = podcasts_df[podcasts_df.is_english == True]
podcasts_df = podcasts_df.drop(columns=['is_english'])
#podcasts_df['episode_titles'], podcasts_df['episode_descriptions'] = podcasts_df.link.apply(get_recent_podcast_episodes)


podcasts_df = podcasts_df.reset_index(drop=True)



podcasts_df.to_pickle('re-scraped_podcasts_with_episode_description.pkl')

                         title  \
0                 Kosmographia   
1      Coffee + Crumbs Podcast   
2                      Drilled   
3  Fantasy Baseball Today in 5   
4              Best Laid Plans   

                                            producer          genre  \
0                                    Randall Carlson        Science   
1          Katie Blackburn, Jill Atogwe, Ashlee Gadd  Kids & Family   
2                                 Critical Frequency        Science   
3  CBS Sports, Fantasy Baseball, MLB, Fantasy Ran...         Sports   
4                                   Sarah Hart-Unger      Education   

                                         description  num_episodes  rating  \
0                     All the Randall you can handle            95     4.9   
1  The Coffee + Crumbs Podcast is your companion ...           141     4.8   
2  A true-crime podcast about climate change, hos...           183     4.6   
3  Want Fantasy advice, news and analysis fast? F...    

In [151]:
#fixing typo in episode_description column
loaded = pd.read_pickle('re-scraped_podcasts_with_episode_description.pkl')
loaded = loaded.rename(columns={'epidsode_description':'episode_description'})
loaded.to_pickle('re-scraped_podcasts_with_episode_description.pkl')

In [165]:
loaded.episode_description[0]

["We want to make these recorded podcast more directly about RC research, but you’ll see that even that plan gets derailed! The Carbon Cycle and Carbon “sinks” are revisited, as RC gets into an overview of his detailed article “Redemption of the Beast” that he compiled in 2017, which includes numerous quotes and articles about the absolutely vital role of Carbon Dioxide in the biosphere. He digresses, however, in an attempt to defend his friend Graham Hancock, and ends up in a tussle with Normal Guy Mike. First episode of “Randall Rants” to be extracted from this classic RC diatribe? Kosmographia Ep094 The Randall Carlson Podcast with Brothers of the Serpent – Kyle and Russ, Normal Guy Mike, and GeocosmicREX admin Bradley, from 1/10/23.\n In the name of liberty and freedom, we are moving this podcast to our new partner platform! Please join us here: https://www.howtube.com/channels/RandallCarlson\n LINKS:\n Announcements about events and tours: https://randallcarlson.com/tours-and-even

In [167]:
loaded2 = loaded.episode_description.apply(clean_description)


In [168]:
loaded2[0]

['we want to make these recorded podcast more directly about rc research but youll see that even that plan gets derailed the carbon cycle and carbon sinks are revisited as rc gets into an overview of his detailed article redemption of the beast that he compiled in  which includes numerous quotes and articles about the absolutely vital role of carbon dioxide in the biosphere he digresses however in an attempt to defend his friend graham hancock and ends up in a tussle with normal guy mike first episode of randall rants to be extracted from this classic rc diatribe kosmographia ep the randall carlson podcast with brothers of the serpent  kyle and russ normal guy mike and geocosmicrex admin bradley from   in the name of liberty and freedom we are moving this podcast to our new partner platform please join us here httpswwwhowtubecomchannelsrandallcarlson  links  announcements about events and tours httpsrandallcarlsoncomtoursandevents  rcs monthly updates on science news and his activities

In [79]:
get_recent_podcast_episodes(sample_podcast['link'])

AttributeError: ignored

In [80]:
reload_mine = pd.read_pickle('re-scraped_podcasts.pkl')
reload_mine

,title,producer,genre,description,num_episodes,rating,num_reviews,link,itunes_id
0,Kosmographia,Randall Carlson,Science,All the Randall you can handle,95,4.9,699.0,https://podcasts.apple.com/us/podcast/kosmogra...,1479346591
1,Coffee + Crumbs Podcast,"Katie Blackburn, Jill Atogwe, Ashlee Gadd",Kids & Family,The Coffee + Crumbs Podcast is your companion ...,141,4.8,1700.0,https://podcasts.apple.com/us/podcast/coffee-c...,1093725475
2,Drilled,Critical Frequency,Science,"A true-crime podcast about climate change, hos...",183,4.6,2100.0,https://podcasts.apple.com/us/podcast/drilled/...,1439735906
3,Best Laid Plans,Sarah Hart-Unger,Education,Welcome to Best Laid Plans -- a podcast all ab...,139,4.9,535.0,https://podcasts.apple.com/us/podcast/best-lai...,1525311647
4,The Sports Junkies,Audacy,Sports,"Cakes, Bish, EB and JP on 106.7 the Fan.",500,4.4,1500.0,https://podcasts.apple.com/us/podcast/the-spor...,367825854
...,...,...,...,...,...,...,...,...,...
4107,Oxventure - A Dungeons & Dragons Podcast,Oxventure,Leisure,The Oxventurers Guild presents a DnD real play...,134,5.0,655.0,https://podcasts.apple.com/us/podcast/oxventur...,1528568221
4108,Future of Infrastructure,Microsoft Podcasts,Technology,"Host Jeremy Goldberg, Worldwide Director of Cr...",15,NaN,NaN,https://podcasts.apple.com/us/podcast/future-o...,1674271676
4109,The TWIML AI Podcast (formerly This Week in Ma...,Sam Charrington,Technology,Machine learning and artificial intelligence a...,640,4.7,370.0,https://podcasts.apple.com/us/podcast/the-twim...,1116303051
4110,The Bestie Breakdown,The Bestie Breakdown,Kids & Family,Hi! We're Shay Shull from Mix and Match Mama a...,51,3.8,1400.0,https://podcasts.apple.com/us/podcast/the-best...,1509081920


In [81]:
reload_his = pd.read_pickle('podcast_dataframe_from_siddgood.pkl')
reload_his

,title,producer,genre,description,num_episodes,rating,num_reviews,link,episode_titles,episode_descriptions
0,History Hyenas with Chris Distefano and Yannis...,RiotCast Network,Comedy,Comedians Yannis Pappas and Chris Distefano go...,98.0,4.9,1300.0,https://podcasts.apple.com/us/podcast/history-...,robert moses was wild kinsey was wild prohibit...,the boys discuss robert moses wild the hyenas ...
1,Curiosity Daily,Westwood One,Education,The award-winning Curiosity Daily podcast from...,300.0,4.8,428.0,https://podcasts.apple.com/us/podcast/curiosit...,maps as weapons and art w justin fornal and em...,explorers justin fornal and emiliano ruprah fr...
2,Spirits,Multitude,History,"A boozy weekly podcast about mythology, legend...",161.0,4.8,1200.0,https://podcasts.apple.com/us/podcast/spirits/...,jacheongbi multimedia mythmaking with carmen m...,this story seems like we lifted it straight ou...
3,The Soundtrack Show,iHeartRadio,TV & Film,The Soundtrack Show is a weekly look at film s...,67.0,4.9,879.0,https://podcasts.apple.com/us/podcast/the-soun...,season intermission the nightmare before chri...,season intermission learn more about your ad...
4,Writing Excuses,"Brandon Sanderson, Mary Robinette Kowal, Dan W...",Arts,"Brandon Sanderson, Mary Robinette Kowal, Howar...",58.0,4.7,775.0,https://podcasts.apple.com/us/podcast/writing-...,economics realism vs ruleofcool sequencing you...,your hosts brandon dan howard and mahtab econ...
...,...,...,...,...,...,...,...,...,...,...
4298,Dark Histories,Ben Cutmore,History,Bi-weekly narratives on the unsolved and the u...,72.0,4.9,287.0,https://podcasts.apple.com/us/podcast/dark-his...,the yatton demoniac george lukins the sarah du...,sir when you can spare room in your gazette i ...
4299,Red Scare,Red Scare,Society & Culture,Red Scare is a cultural commentary podcast hos...,99.0,4.0,1200.0,https://podcasts.apple.com/us/podcast/red-scar...,bore carnival teaser pest control greta fg thu...,the ladies discuss the online ya wars the cuts...
4300,The Amelia Project,Imploding Fictions,Fiction,The Amelia Project is a secret agency that fak...,39.0,4.9,386.0,https://podcasts.apple.com/us/podcast/the-amel...,richard season finale ant cleo cole niseag b...,i always obey the dice episode season fina...
4301,L.A. Not So Confidential,Dr. Scott & Dr. Shiloh,True Crime,An L.A. based podcast brought to you by two fo...,36.0,4.8,151.0,https://podcasts.apple.com/us/podcast/l-a-not-...,killer nurses the fear of clowns intimate part...,dr scott and dr shiloh delve into the world of...


In [87]:
reload_his.iloc[1].episode_descriptions

'explorers justin fornal and emiliano ruprah from the science channels unexplained and unexplored discuss the surprising roles that maps have played throughout history youll also learn about why people fidget learn about whether humans have an internal compass how long a flu virus can last in the environment and whether creativity and mental illness actually go hand in hand forrest galante host of extinct or alive on animal planet explains how to become an adventurer survive extreme conditions and find extinct animals then youll learn about how cryptomnesia can make you plagiarize without realizing it learn about why the amount of light you see affects how cold you feel whats inside a kangaroos pouch and foods that can curb the effects of sleep deprivation popular mathematics writer and cornell university professor steven strogatz discusses the difference between calculus and algebra and what calculus can do that other math cant plus learn about how distraction can alter your memory an